## Setup

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import requests
import pandas as pd
import numpy as np
from datetime import datetime
import geopandas as gpd
import pygeos

## Township/range geo file
Data is grouped into township/range regions for the maps.

In [3]:
# load the plss shapefile (these only include TRS areas that are within the San Joaquin subbasin)
plss = gpd.read_file("./data/plss_subbasin.geojson")

In [4]:
plss.sample()

,OBJECTID,Township,Range,Meridian,Source,Section,MTRS,TownshipRange,geometry
10609,74136,T23S,R18E,MDM,BLM,28,MDM-T23S-R18E-28,T23S R18E,"MULTIPOLYGON (((-120.02203 35.90592, -120.0264..."


In [5]:
# create merged range geofile
plss_range = plss.dissolve(by='TownshipRange').reset_index()

In [6]:
plss_range.sample()

,TownshipRange,geometry,OBJECTID,Township,Range,Meridian,Source,Section,MTRS
331,T20S R25E,"POLYGON ((-119.28626 36.13831, -119.29044 36.1...",65666,T20S,R25E,MDM,BLM,1,MDM-T20S-R25E-1


## Well completion reports

https://data.cnra.ca.gov/dataset/well-completion-reports

### Get the data

In [7]:
def search_query(query):
    api = requests.get('https://data.cnra.ca.gov/api/3/action/datastore_search?resource_id=8da7b93b-4e69-495d-9caa-335691a1896b&limit=32000&q='+query).json()
    data = api['result']['records']
    while api['result']['records']:
        api = requests.get('https://data.cnra.ca.gov'+api['result']['_links']["next"]).json()
        data.extend(api['result']['records'])
    return data

In [8]:
# grab records from all counties that overlap with the san joaquin valley basin
df_san_joaquin = pd.DataFrame(data=search_query('san+joaquin'))
df_kings = pd.DataFrame(data=search_query('kings'))
df_stanislaus = pd.DataFrame(data=search_query('stanislaus'))
df_merced = pd.DataFrame(data=search_query('merced'))
df_fresno = pd.DataFrame(data=search_query('fresno'))
df_madera = pd.DataFrame(data=search_query('madera'))
df_tulare = pd.DataFrame(data=search_query('tulare'))
df_kern = pd.DataFrame(data=search_query('kern'))
# additional counties that are in the SJV subbasin
df_sacramento = pd.DataFrame(data=search_query('sacramento'))
df_calaveras = pd.DataFrame(data=search_query('calaveras'))
df_mariposa = pd.DataFrame(data=search_query('mariposa'))
df_tuolumne = pd.DataFrame(data=search_query('tuolumne'))
df_contra_costa = pd.DataFrame(data=search_query('contra+costa'))
df_san_benito = pd.DataFrame(data=search_query('san+benito'))
df_alameda = pd.DataFrame(data=search_query('alameda'))
df_amador = pd.DataFrame(data=search_query('amador'))

In [9]:
# combine county datasets
completion = pd.concat([df_san_joaquin, df_kings, df_stanislaus, df_merced, df_fresno, df_madera, df_tulare, df_kern, 
                        df_sacramento, df_calaveras, df_mariposa, df_tuolumne, df_contra_costa, df_san_benito, df_alameda, df_amador])

### Clean the data

In [10]:
# remove duplicates
completion.drop_duplicates(inplace=True)

In [11]:
# remove rows not in the counties we want
completion = completion.loc[completion['COUNTYNAME'].isin(['San Joaquin', 'Kings', 'Stanislaus', 'Merced', 'Fresno', 'Madera', 'Tulare', 'Kern',
                                                           'Sacramento', 'Calaveras', 'Mariposa', 'Tuolumne', 'Contra Costa', 'San Benito', 'Alameda', 'Amador'])]

In [12]:
completion.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 337294 entries, 0 to 8490
Data columns (total 47 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   DECIMALLATITUDE               316089 non-null  object 
 1   WORKFLOWSTATUS                19692 non-null   object 
 2   LLACCURACY                    289527 non-null  object 
 3   PERMITDATE                    18971 non-null   object 
 4   PUMPTESTLENGTH                18628 non-null   object 
 5   SECTION                       322237 non-null  object 
 6   rank                          337294 non-null  float64
 7   REGIONOFFICE                  337294 non-null  object 
 8   DRILLINGMETHOD                193007 non-null  object 
 9   OTHEROBSERVATIONS             21484 non-null   object 
 10  WELLYIELDUNITOFMEASURE        63915 non-null   object 
 11  WELLLOCATION                  163415 non-null  object 
 12  PERMITNUMBER                  87301 non-null  

In [13]:
# filter to just the columns we are interested in
completion_trim = completion[["DECIMALLATITUDE", "DECIMALLONGITUDE", "TOWNSHIP", "RANGE", "SECTION", "WELLLOCATION", "CITY", "COUNTYNAME", 
                              "PERMITNUMBER", "BOTTOMOFPERFORATEDINTERVAL", "TOPOFPERFORATEDINTERVAL", "GROUNDSURFACEELEVATION", "STATICWATERLEVEL", 
                              "RECORDTYPE",  "PLANNEDUSEFORMERUSE", "LOCALPERMITAGENCY", "WCRNUMBER", "TOTALDRILLDEPTH", 
                              "TOTALCOMPLETEDDEPTH", "DATEWORKENDED", "DRILLERNAME", "DRILLERLICENSENUMBER", "CASINGDIAMETER"]]

In [14]:
# rename columns
completion_trim.rename(columns={"DECIMALLATITUDE" : "latitude", 
                                "DECIMALLONGITUDE" : "longitude", 
                                "SECTION" : "section", 
                                "TOWNSHIP" : "township", 
                                "RANGE" : "range", 
                                "WELLLOCATION" : "location", 
                                "CITY" : "city", 
                                "COUNTYNAME" : "county", 
                                "PERMITNUMBER" : "permit_no", 
                                "BOTTOMOFPERFORATEDINTERVAL" : "bottom_perf_interval", 
                                "TOPOFPERFORATEDINTERVAL" : "top_perf_interval", 
                                "GROUNDSURFACEELEVATION" : "gse", 
                                "STATICWATERLEVEL" : "static_water_level", 
                                "RECORDTYPE" : "type", 
                                "PLANNEDUSEFORMERUSE" : "use", 
                                "LOCALPERMITAGENCY" : "permit_agency", 
                                "WCRNUMBER" : "wcr_no", 
                                "TOTALDRILLDEPTH" : "drill_depth", 
                                "TOTALCOMPLETEDDEPTH" : "completed_depth", 
                                "DATEWORKENDED" : "date_work_ended", 
                                "DRILLERNAME" : "driller", 
                                "DRILLERLICENSENUMBER": "driller_license_no",
                                "CASINGDIAMETER" : "casing_diameter"
                               }, inplace=True)

In [15]:
# filter to only include new well completion
completion_trim = completion_trim.loc[completion_trim['type'] == 'WellCompletion/New/Production or Monitoring/NA']

In [16]:
# filter to only include agriculture, domestic, or public wells
completion_trim['use'] = completion_trim['use'].apply(lambda x: "" if x is None else x)
completion_trim['use'] = completion_trim['use'].apply(lambda x: "agriculture" if "Agri" in x else 
                                                                "domestic" if "Domestic" in x else
                                                                "public" if "Public" in x else "Other")
completion_trim = completion_trim[completion_trim["use"].isin(["agriculture","domestic","public"])]

In [17]:
# convert depth to number
completion_trim['completed_depth'] = pd.to_numeric(completion_trim['completed_depth'], errors="coerce")
# removes depth data that is incorrect in new column
completion_trim['completed_depth_updated'] = completion_trim['completed_depth'].apply(lambda x: x if x >= 20 else np.nan)

In [18]:
# convert date work ended to datetime and filter to only include completed dates that are possible (not a future date) 
completion_trim['date_work_ended'] = pd.to_datetime(completion_trim['date_work_ended'], errors='coerce')
completion_trim['date_work_ended_updated'] = completion_trim['date_work_ended'].apply(lambda x: x if x < datetime.now() else np.nan)
# create simple year and month columns
completion_trim['year_work_ended'] = pd.DatetimeIndex(completion_trim['date_work_ended_updated']).year
completion_trim['month_work_ended'] = pd.DatetimeIndex(completion_trim['date_work_ended_updated']).month

In [19]:
# clean longitude/latitude data
completion_trim['longitude'] = completion_trim['longitude'].replace(to_replace= r'\/', value= '', regex=True)
completion_trim['latitude'] = completion_trim['latitude'].replace(to_replace= r'\/', value= '', regex=True)

In [20]:
# create wells geodataframe
gdf_wells = gpd.GeoDataFrame(completion_trim, geometry=gpd.points_from_xy(completion_trim.longitude, completion_trim.latitude))
gdf_wells = gdf_wells.dropna(subset=['latitude','longitude'])
gdf_wells = gdf_wells.set_crs('epsg:4326')

In [21]:
# match up based on longitude/latitude
join_wells_plss = gdf_wells.sjoin(plss, how="left")

In [22]:
# drop the ones that aren't in the san joaquin valley basin
join_wells_plss = join_wells_plss.dropna(subset=['MTRS'])

In [23]:
join_wells_plss.sample()

,latitude,longitude,township,range,section,location,city,county,permit_no,bottom_perf_interval,...,geometry,index_right,OBJECTID,Township,Range,Meridian,Source,Section,MTRS,TownshipRange
18071,36.94503,-119.903,11S,19E,28,AVE 14 E OF RD 35,MADERA,Madera,9371,400,...,POINT (-119.90300 36.94503),5345.0,39225.0,T11S,R19E,MDM,BLM,28.0,MDM-T11S-R19E-28,T11S R19E


In [24]:
join_wells_plss.drop_duplicates(inplace=True)

In [25]:
# Save dataframe
join_wells_plss.to_csv("./data/well_completion.csv", index=False)

## Shortage reports
https://data.ca.gov/dataset/household-water-supply-shortage-reporting-system-data

### Get the data

In [26]:
shortage_api = requests.get('https://data.cnra.ca.gov/api/3/action/datastore_search?resource_id=e1fd9f48-a613-4567-8042-3d2e064d77c8&limit=32000').json()
data_shortage = shortage_api['result']['records']
while shortage_api['result']['records']:
    shortage_api = requests.get('https://data.cnra.ca.gov'+shortage_api['result']['_links']["next"]).json()
    data_shortage.extend(shortage_api['result']['records'])

In [27]:
df_shortage = pd.DataFrame(data=data_shortage)

### Clean the data

In [28]:
len(df_shortage)

3753

In [29]:
# Convert date to date time
df_shortage['CreateDateUpdated'] = pd.to_datetime(df_shortage['CREATE DATE'], errors='coerce')

# Create a simple year and month that report was created
df_shortage['CreateDateYear'] = pd.DatetimeIndex(df_shortage['CreateDateUpdated']).year
df_shortage['CreateDateMonth'] = pd.DatetimeIndex(df_shortage['CreateDateUpdated']).month

In [30]:
df_shortage = df_shortage.dropna(subset=['LATITUDE','LONGITUDE'])

In [31]:
gdf_shortage = gpd.GeoDataFrame(df_shortage, geometry=gpd.points_from_xy(df_shortage.LONGITUDE, df_shortage.LATITUDE))

In [32]:
gdf_shortage = gdf_shortage.set_crs('epsg:4326')

In [33]:
join_shortage_plss = gdf_shortage.sjoin(plss, how="left")
join_shortage_plss.sample()

,Well Depth,StatusType,Status,Household Support,Pump Rate Reduction,Water Issues,Shortage Type,Was Issue Resolved?,CREATE DATE,Approximate Issue Start Date,...,geometry,index_right,OBJECTID,Township,Range,Meridian,Source,Section,MTRS,TownshipRange
2211,Dont know,Resolved,Resolved,None,Not sure,"Reduction in water pressure, lower flows.\r\n,...",None,"Yes, lowered the pump bowl\r\n",08/26/2020,08/06/2020,...,POINT (-122.35051 41.60554),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# drop the ones that aren't in a subbasin trs
join_shortage_plss = join_shortage_plss.dropna(subset=['MTRS'])

In [35]:
len(join_shortage_plss)

2519

In [36]:
# save dataframe
join_shortage_plss.to_csv("./data/shortages.csv", index=False)

## Groundwater measurements
https://data.cnra.ca.gov/dataset/periodic-groundwater-level-measurements

### Get the data

In [37]:
measurements_api = requests.get('https://data.cnra.ca.gov/api/3/action/datastore_search?resource_id=bfa9f262-24a1-45bd-8dc8-138bc8107266&limit=32000').json()
data_measurements = measurements_api['result']['records']
while measurements_api['result']['records']:
    measurements_api = requests.get('https://data.cnra.ca.gov'+measurements_api['result']['_links']["next"]).json()
    data_measurements.extend(measurements_api['result']['records'])

In [38]:
# select data from result section
df_measurements = pd.DataFrame(data=data_measurements)

In [39]:
df_measurements.sample()

,GWE,WLM_ORG_NAME,COOP_ORG_NAME,WLM_ACC_DESC,GSE_GWE,MSMT_DATE,SITE_CODE,WLM_GSE,MSMT_CMT,WLM_ID,MONITORING_PROGRAM,WLM_QA_DESC,_id,WLM_DESC,WLM_RPE
694251,193.7,Department of Water Resources,Department of Water Resources,Water level accuracy is unknown,79.0,1970-03-20T00:00:00,353842N1194585W001,272.7,None,734846,VOLUNTARY,None,694198,Unknown,273.7


In [40]:
# stations data has the location info for the measurement wells
stations_api = requests.get('https://data.cnra.ca.gov/api/3/action/datastore_search?resource_id=af157380-fb42-4abf-b72a-6f9f98868077&limit=32000').json()
data_stations = stations_api['result']['records']
while stations_api['result']['records']:
    stations_api = requests.get('https://data.cnra.ca.gov'+stations_api['result']['_links']["next"]).json()
    data_stations.extend(stations_api['result']['records'])

In [41]:
df_stations = pd.DataFrame(data=data_stations)

In [42]:
simple_station_data = df_stations.loc[:,['SITE_CODE','LONGITUDE','LATITUDE','GSE','COUNTY_NAME','WELL_USE']]

In [43]:
simple_station_data.sample()

,SITE_CODE,LONGITUDE,LATITUDE,GSE,COUNTY_NAME,WELL_USE
2947,334339N1171412W003,-117.141,33.4339,1142.3,Riverside,Unknown


### Clean the data

In [44]:
# create simple year and month columns
df_measurements['year'] = pd.DatetimeIndex(df_measurements['MSMT_DATE']).year
df_measurements['month'] = pd.DatetimeIndex(df_measurements['MSMT_DATE']).month

In [45]:
# filter for records that have the ground surface elevation to groundwater elevation measurements
df_measurements = df_measurements[~pd.isna(df_measurements['GSE_GWE'])]

In [46]:
# filter for just the spring measurements
spring = [1,2,3,4]
spring_measurements = df_measurements[df_measurements['month'].isin(spring)]

In [47]:
# merge with station data for location info
measurements_merge = spring_measurements.merge(simple_station_data, on='SITE_CODE')
measurements_merge.sample()

,GWE,WLM_ORG_NAME,COOP_ORG_NAME,WLM_ACC_DESC,GSE_GWE,MSMT_DATE,SITE_CODE,WLM_GSE,MSMT_CMT,WLM_ID,...,_id,WLM_DESC,WLM_RPE,year,month,LONGITUDE,LATITUDE,GSE,COUNTY_NAME,WELL_USE
665194,145.0,Department of Water Resources,Yolo County Flood Control and Water Conservati...,Water level accuracy is unknown,62.6,1990-03-19T00:00:00,387506N1220252W001,207.6,None,2773039,...,1808127,Unknown,208.6,1990,3,-122.025,38.7506,208.79,Yolo,Irrigation


In [48]:
# drop the rows that have incorrect measurements that of 0 or less
measurements_merge = measurements_merge[measurements_merge['GSE_GWE'] > 0]

In [49]:
gdf_measurements = gpd.GeoDataFrame(
    measurements_merge, 
    geometry=gpd.points_from_xy(
        measurements_merge.LONGITUDE, 
        measurements_merge.LATITUDE
    ))

In [50]:
gdf_measurements = gdf_measurements.set_crs('epsg:4326')

In [51]:
# match up based on longitude/latitude
join_measurements_plss = gdf_measurements.sjoin(plss, how="left")
join_measurements_plss.sample()

,GWE,WLM_ORG_NAME,COOP_ORG_NAME,WLM_ACC_DESC,GSE_GWE,MSMT_DATE,SITE_CODE,WLM_GSE,MSMT_CMT,WLM_ID,...,geometry,index_right,OBJECTID,Township,Range,Meridian,Source,Section,MTRS,TownshipRange
620036,58.78,Sonoma County PRMD,Sonoma County PRMD,Water level accuracy to nearest tenth of a foot,33.9,2011-01-09T00:00:00,384230N1228158W001,92.68,None,1832050,...,POINT (-122.81600 38.42300),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
# drop the ones that aren't in a subbasin trs
join_measurements_plss = join_measurements_plss.dropna(subset=['MTRS'])

In [53]:
join_measurements_plss.sample()

,GWE,WLM_ORG_NAME,COOP_ORG_NAME,WLM_ACC_DESC,GSE_GWE,MSMT_DATE,SITE_CODE,WLM_GSE,MSMT_CMT,WLM_ID,...,geometry,index_right,OBJECTID,Township,Range,Meridian,Source,Section,MTRS,TownshipRange
418044,257.95,Department of Water Resources,Panoche Water District,Water level accuracy is unknown,8.95,2006-04-20T00:00:00,367855N1206479W002,266.9,None,3032730,...,POINT (-120.64800 36.78550),6016.0,44915.0,T13S,R12E,MDM,BLM,22.0,MDM-T13S-R12E-22,T13S R12E


In [54]:
site_lon_lat = join_measurements_plss[["SITE_CODE","LONGITUDE","LATITUDE"]]
site_lon_lat.columns = ["site","longitude","latitude"]
site_lon_lat.drop_duplicates(subset="site", inplace=True)
site_lon_lat.sample()

,site,longitude,latitude
507789,372438N1206341W001,-120.634,37.2438


In [55]:
# Group wells that had multiple spring measurements in some years and get the average
measurements_plss_group = join_measurements_plss.groupby(['SITE_CODE','MTRS','TownshipRange','COUNTY_NAME','year']).agg({
    'GSE_GWE': ['mean'],
}).reset_index()
measurements_plss_group.columns = ['site','MTRS','TownshipRange','county','year','gse_gwe']

In [56]:
measurements_plss_group.head()

,site,MTRS,TownshipRange,county,year,gse_gwe
0,344779N1192479W001,MDM-T28S-R25E-23,T28S R25E,Kern,2012,254.0
1,344779N1192479W001,MDM-T28S-R25E-23,T28S R25E,Kern,2013,262.0
2,344779N1192479W001,MDM-T28S-R25E-23,T28S R25E,Kern,2014,272.0
3,344779N1192479W001,MDM-T28S-R25E-23,T28S R25E,Kern,2015,280.0
4,344779N1192479W001,MDM-T28S-R25E-23,T28S R25E,Kern,2016,292.0


In [57]:
# export all measurements for analysis
measurements_plss_group.to_csv("./data/spring_water_levels.csv", index=False)